In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
#import matplotlib.patches as patches
from matplotlib import ticker
#from mpl_toolkits.mplot3d import Axes3D
import datetime
import pandas as pd
import time
import os
from os import listdir
from os.path import join, exists, normpath, isdir, isfile
import CubicSplineTrack
#(c)2015 Robert Sedgewick, Kevin Wayne, and Robert Dondero
from greatcircle import great_circle
from datetime import date
#import shutil

from matplotlib.ticker import Formatter
from mpl_toolkits.axisartist.parasite_axes import HostAxes, ParasiteAxes
#Turn off interactive plotting for pyplot
plt.ioff()

In [ ]:
"""Team WAAAAAAEEEEEEEEEE Bhargav"""
#Creating intensity table
intensity_values = {
     'T-Number': [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5],
  '1-min Winds': [25, 30, 35, 45, 55, 65, 77, 90, 102, 115, 127, 140, 155, 170, 185],
'Min. Pressure': [0, 1009, 1005, 1000, 994, 987, 979, 970, 960, 948, 935, 921, 906, 890, 873]
}
T_frame = pd.DataFrame(intensity_values)

def Tvalue(wind_speed):
    i=0
    current = 0
    prev=0
    after=0
    for i in range(15):
        current = T_frame.loc[i, '1-min Winds']
        if (i == 0):
            prev=20
        else:
            prev = T_frame.loc[(i-1), '1-min Winds']
        
        if(i==14):
            after = 188
            
        else:
            after=T_frame.loc[(i+1), '1-min Winds']
        
        if(wind_speed > 185):
            return 8.5
        
        else:
            half_prev =current - ((current-prev)/2)
            half_aft = current + ((after-prev)/2)
            if(wind_speed >= half_prev and wind_speed < half_aft):
                return T_frame.loc[i, 'T-Number']
    return 0

"""Lance"""
def run_circle(row, center_lat, center_lon): 
    lat = (row['Lat'])
    lon = (row['Long'])
    return great_circle(lat, lon, center_lat, center_lon)

"Lance" 
def get_glm_data_path(day, hour, start_minute, base_path):
    temp_path = base_path + '/' + str(day)
    #print(temp_path)

    start_string = "0.0"
    hour_string = ""
    #end_string = "__.csv"
    if hour < 10:
        hour_string = f"0{hour}"
    else:
        hour_string = f"{hour}"
    temp_path += '/' + hour_string
    
    files_in_path = listdir(temp_path)
    #print(files_in_path)
    file_found = False
    for file in files_in_path:
        if file.startswith(start_string):
            temp_path = join(temp_path +'/', file)
            file_found = True
    
    if not file_found:
        return ""
    
    return temp_path

"Bhargava's Code"
#Function to get and arrange generated csv file
def get_csv(csv_file_path):
    temp_csv = pd.read_csv(csv_file_path)
    temp_csv["Date"] = temp_csv["Date"].apply(pd.to_datetime)
    temp_csv= temp_csv.sort_values(["Date"], ascending=True)
    if len(temp_csv["Date"]) == 0:
        return temp_csv
    index = pd.Series(range(int(temp_csv.index.max())+1))
    temp_csv=temp_csv.set_index(index)
    return temp_csv

"Cassie"
def load_glm_time_range(glm_path, date_val, minuite_interval, glm_interval):
    #print(date_val)
    day = date_val.timetuple().tm_yday
    #print(get_glm_data_path(day, date_val.hour, date_val.minute, glm_paths[storm_index]))
    data_out = [get_csv(get_glm_data_path(day, date_val.hour, date_val.minute, glm_path))]
    indx = glm_interval

    if glm_interval >= minuite_interval:
        return data_out[0]

    new_date = date_val
    while indx < minuite_interval:
        new_date = new_date + datetime.timedelta(minutes=minuite_interval)
        day = (new_date - datetime.datetime(new_date.year, 1, 1)).days
        data_out.append(get_csv(get_glm_data_path(day, new_date.hour, new_date.minute, glm_path)))
        indx += glm_interval
    
    return pd.concat(data_out)

In [ ]:
"""
TODO
-Font BIGGER
-Seperate Inner & Outer ? maybe?
-Tick mark stuff?
"""
class MyFormatter(Formatter):
    def __init__(self, dates):
        self.dates = dates
        self.index = 0
        self.freq = int(len(self.dates) / 10)

    def __call__(self, x, pos=0):
        ind = int(round(x))
        if ind >= len(self.dates) or ind < 0:
            return ''
        #return self.dates[ind]
        return f'{self.dates[ind].month}/{self.dates[ind].day}'

def plot_data(wwlln, glm, dates, save_path, name, modify):
    plt.rcParams.update({'font.size': 18})
    x = np.arange(len(dates))
    
    wwlln_text = 'WWLLN Strokes'
    if not modify == 1:
        wwlln_text = f'WWLLN Strokes * {modify}'

    width = 1
    fig, ax = plt.subplots()
    rects1 = ax.bar(x, glm, width, label='GLM Groups')
    rects2 = ax.bar(x, wwlln, width, label=wwlln_text)

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Strike & Group Count')
    ax.set_title(f'WWLLN Strikes per Hour vs GLM Groups per Hour\n{name} Inner Core (100km) & Rain Bands (200km-400km)')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(int(len(dates) / 10)))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(int(len(dates) / 50)))
    ax.xaxis.set_major_formatter(MyFormatter(dates))
    ax.tick_params(which='major', length=10)
    ax.tick_params(which='minor', length=5)
    fig.autofmt_xdate()
    ax.legend()

    #ax.bar_label(rects1, padding=3)
    #ax.bar_label(rects2, padding=3)

    fig.set_size_inches(18.5, 10.5)
    fig.tight_layout()
    plt.savefig(save_path + name + " Histogram.png", bbox_inches='tight', pad_inches=.4, dpi=100)
    return

def plot_with_MORE_AXIES(wwlln, glm, pressure, wind_speed, dates, save_path, name, modify):
    plt.rcParams.update({'font.size': 18})
    x = np.arange(len(dates))
    
    wwlln_text = 'WWLLN Strikes'
    if not modify == 1:
        wwlln_text = f'WWLLN Strikes * {modify}'

    width = 1
    fig = plt.figure()

    host = fig.add_axes([0.15, 0.1, 0.65, 0.8], axes_class=HostAxes)
    par1 = ParasiteAxes(host, sharex=host)
    par2 = ParasiteAxes(host, sharex=host)
    host.parasites.append(par1)
    host.parasites.append(par2)

    host.axis["right"].set_visible(False)
    host.axis["top"].set_visible(False)

    par1.axis["right"].set_visible(True)
    par1.axis["right"].major_ticklabels.set_visible(True)
    par1.axis["right"].label.set_visible(True)
    par2.axis["right2"] = par2.new_fixed_axis(loc="right", offset=(100, 0))

    p2, = par1.plot(x, pressure, label="Pressure (mb)", color='g')
    p3, = par2.plot(x, wind_speed, label="Sustained Wind (knots)", color='r')

    par1.set_ylim(825, 1025)
    par2.set_ylim(0, 200)
    
    host.set_xlabel("Date")
    host.set_ylabel("Strike & Group Count")
    par1.set_ylabel("Pressure (mb)")
    par2.set_ylabel("Sustained Wind (knots)")

    par1.axis["right"].label.set_color(p2.get_color())
    par2.axis["right2"].label.set_color(p3.get_color())

    rects1 = host.bar(x, glm, width, label='GLM Groups')
    rects2 = host.bar(x, wwlln, width, label=wwlln_text)

    # Add some text for labels, title and custom x-axis tick labels, etc.
    host.set_ylabel('Strike & Group Count')
    host.set_title(f'WWLLN Strikes per Hour vs GLM Groups per Hour\n{name} Inner Core (100km) & Rain Bands (200km-400km)')
    host.xaxis.set_major_locator(ticker.MultipleLocator(int(len(dates) / 10)))
    host.xaxis.set_minor_locator(ticker.MultipleLocator(int(len(dates) / 50)))
    host.xaxis.set_major_formatter(MyFormatter(dates))
    host.tick_params(which='major', length=10)
    host.tick_params(which='minor', length=5)
    #fig.autofmt_xdate()
    host.legend()

    #ax.bar_label(rects1, padding=3)
    #ax.bar_label(rects2, padding=3)

    fig.set_size_inches(18.5, 10.5)
    fig.tight_layout()
    plt.savefig(save_path + name + " Histogram.png", bbox_inches='tight', pad_inches=.4, dpi=100)
    return

In [ ]:
"""
This code has been re-shaped to only work with glm data
Credit to team squeem and Cassie, Lance, Bhargav

Plots data in every minuite_interval of time from the start to the end dates of a given storm
Does DBSCAN, core and rainband split, and T value calculation

wwlln_track_path: Type: String, .txt File Type
The path (including file name and extension) to the wwlln track file. Only really needed for Dates and Tvalue calculations

cubic_spline_track_path: Type: String, .csv File Type
The path (including file name and extension) to the cubic spline track file.
If code is needed to generate this, please contact Cassie (c.treece@digipen.edu) or Ben (ben.vanoostendorp@digipen.edu)

glm_csv_floder_path: Type: String, path to a folder of the format [folder] -> days -> hours -> data
The path to the glm csv data, must contain the days and hours that the storm was active

Image_output_path: Type: String, will result in a .png File Type
The path to the FOLDER in which the images will be saved. Must end in a '/'

Storm_Name: Type: String
The Name of the storm, display purposes only
Optional, defaults to an empty string

minuite_interval: Type: Int
The time interval in minutes that the data wil be devided into and plotted

split_storm: Type: Bool
set to False to plot all the data within the minuite interval
by default splits the data into the core and rain bands. Ranges from 0km to 100 km, and 200km to 400km away from the center of the storm
"""

def storm_bar(wwlln_track_path, wwlln_data_path, cubic_spline_track_path, glm_csv_folder_path, Image_output_path, Storm_Name = "", minuite_interval = 60, split_storm = True, glm_interval = 60, modify = 1):
    """Team Squeem"""
    # This loads the WWLLN trackfile Jeremy and Natalia give us
    # wind = nautical mph, pressure = millibars
    wwlln_track = pd.read_csv(wwlln_track_path, header=None, names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"], low_memory=False, sep='\t')
    wwlln_track = wwlln_track.drop("Unused", axis=1)#data filtering
    wwlln_track['Date'] = wwlln_track.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}", format="%Y-%m-%d-%H"), axis=1)
    
    wwlln_data = pd.read_csv(wwlln_data_path, header=None, names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"], low_memory=False, sep=' ')
    wwlln_data['Date'] = wwlln_data.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}-{int(x['Min'])}-{x['Sec']}", format="%Y-%m-%d-%H-%M-%S.%f"), axis=1)#is a data clean

    # This opens the cubic spline trackfiles that Ben creates
    storm_center = pd.read_csv(cubic_spline_track_path)
    storm_center['Date'] = storm_center['Date'].apply(pd.to_datetime)

    # Genereates the list of dates from the WWLLN trackfile of a (minuite_interval) range
    # This could be done with the cubic spline file, but this just happens to be what we use
    dates = pd.date_range(wwlln_track['Date'][0], wwlln_track['Date'][len(wwlln_track)-1], freq=str(minuite_interval)+"T")

    bin_count = len(dates)
    glm_counts = [0] * bin_count
    wwlln_counts = [0] * bin_count

    track_index = 0
    pressure = [0] * bin_count
    wind_speed = [0] * bin_count

    # Counter and iteration
    for index, date_val in enumerate(dates): # this loop makes one image for every (minuite_interval) amount of time
        starttime = time.perf_counter() # speeed timer start
        center = storm_center[storm_center['Date'] == date_val] # storm center at this time frame
        
        #----------------------------------------------------------------------------------------#
        """ This loads the data """
        #----------------------------------------------------------------------------------------#
        glm_data = load_glm_time_range(glm_csv_folder_path, date_val, minuite_interval, glm_interval)



        """Cassie and Lance"""
        if split_storm:
            glm_curr_timeframe = glm_data[(glm_data['Date'] >= date_val) & (glm_data['Date'] < date_val + datetime.timedelta(minutes=minuite_interval))]
            wwlln_curr_timeframe = wwlln_data[(wwlln_data['Date'] >= date_val) & (wwlln_data['Date'] < date_val + datetime.timedelta(minutes=minuite_interval))]
            
            if len(glm_curr_timeframe) != 0:
                glm_curr_timeframe['Distance'] = glm_curr_timeframe.apply(run_circle, axis = 1, args = ((center['Lat']), (center['Long'])))
                glm_subset = glm_curr_timeframe[(glm_curr_timeframe['Distance'] <= 100 / 1.852) | ((glm_curr_timeframe['Distance'] >= 200 / 1.852) & (glm_curr_timeframe['Distance'] <= 400 / 1.852))]
            else:
                glm_subset = glm_curr_timeframe
            
            if len(wwlln_curr_timeframe) != 0:
                wwlln_curr_timeframe['Distance'] = wwlln_curr_timeframe.apply(run_circle, axis = 1, args = ((center['Lat']), (center['Long'])))
                wwlln_subset = wwlln_curr_timeframe[(wwlln_curr_timeframe['Distance'] <= 100 / 1.852) | ((wwlln_curr_timeframe['Distance'] >= 200 / 1.852) & (wwlln_curr_timeframe['Distance'] <= 400 / 1.852))]
            else:
                wwlln_subset = wwlln_curr_timeframe
        else:
            # Select just the lightning locations that are within (check_minutes) minutes
            glm_subset = glm_data[(glm_data['Date'] >= date_val) & (glm_data['Date'] < date_val + datetime.timedelta(minutes=minuite_interval))]
            wwlln_subset = wwlln_data[(wwlln_data['Date'] >= date_val) & (wwlln_data['Date'] < date_val + datetime.timedelta(minutes=minuite_interval))]
        
        glm_counts[index] = len(glm_subset)
        wwlln_counts[index] = len(wwlln_subset) * modify

        if track_index+1 < len(wwlln_track['Date']) and date_val >= wwlln_track['Date'][track_index+1]:
            track_index += 1
        
        pressure[index] = wwlln_track['Min_Pressure'][track_index]
        wind_speed[index] = wwlln_track['Max_Winds'][track_index]

        # Shows roughly how long we have left
        taken = time.perf_counter() - starttime
        print(f"{Storm_Name}: \tPercent Done: {(index + 1) / len(dates) * 100 : .2f}% \tTime taken: {taken : .2f} seconds \tEst time remaining: {datetime.timedelta(seconds=(len(dates) - (index + 1)) * taken)} {glm_counts[index]}", end='\r')
    
    
    plot_with_MORE_AXIES(wwlln_counts, glm_counts, pressure, wind_speed, dates.date, Image_output_path, Storm_Name, modify)


In [ ]:
storm_name = "Dorian"
Cubic_Spline_Path = f"./Data/{storm_name}/{storm_name} Cubic Spline Trackfile.csv"

# this file name is unique to each storm in some situations. for automatically
# getting the file paths, check for "Trackfile.txt" at the end of file names
WWLLN_Track_Path = f"./Data/{storm_name}/ATL_19_5_Dorian_Reduced_Trackfile.txt"
wwlln_data_path = f"./Data/{storm_name}/ATL_19_5_Dorian_WWLLN_Locations.txt"

glm_folder_path = f"./Data/{storm_name}/glm/"
image_out_path = f"./Images/glm_Hurricane {storm_name}/"
check_minutes = 60

if not exists(Cubic_Spline_Path):
    # can take a third interval parameter, defaults to 10 minute intervals of location data
    CubicSplineTrack.cubic_spline_trackfile(WWLLN_Track_Path, Cubic_Spline_Path)

if not os.path.exists(image_out_path):
    os.mkdir(image_out_path)

storm_bar(WWLLN_Track_Path, wwlln_data_path, Cubic_Spline_Path, glm_folder_path, image_out_path, storm_name, modify=10)